# ============================================
# regression code
# ============================================

In [1]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
from statsmodels.tools import add_constant

pd.set_option('display.width', 1000)


# --- Helper: Add significance stars ---
def add_stars(coef, pvalue):
    if pvalue < 0.01:
        return f"{coef:.3f}***"
    elif pvalue < 0.05:
        return f"{coef:.3f}**"
    elif pvalue < 0.1:
        return f"{coef:.3f}*"
    else:
        return f"{coef:.3f}"
# ---  R² ---
def R_squared(result, y):
    ssr = (result.resids ** 2).sum()
    tss = ((y - y.mean()) ** 2).sum()
    return 1 - ssr / tss# ---  R² ---
# --- Build custom table with ALL variables including controls ---
def build_regression_table(results, model_data, sample_info, var_labels, model_keys, col_names, controls, output_file):
    
    # Ensure all variables have labels
    all_vars = set()
    for res in results.values():
        all_vars.update(res.params.index)
    
    for var in all_vars:
        if var not in var_labels:
            var_labels[var] = var

    # Define desired order
    main_vars_order = [v for v in var_labels if v in all_vars and v not in controls and v != 'const']
    control_vars_order = [v for v in controls if v in all_vars]
    const_var = ['const'] if 'const' in all_vars else []

    ordered_vars = main_vars_order + control_vars_order + const_var

    # Build table rows
    table_rows = []
    for var in ordered_vars:
        coef_row = {'Variables': var_labels[var]}
        se_row = {'Variables': ''}
        for key in model_keys:
            res = results[key]
            if var in res.params.index:
                coef = res.params[var]
                pval = res.pvalues[var]
                se = res.std_errors[var]
                coef_row[key] = add_stars(coef, pval)
                se_row[key] = f"({se:.3f})"
            else:
                coef_row[key] = ""
                se_row[key] = ""
        table_rows.extend([coef_row, se_row])

    df_table = pd.DataFrame(table_rows)

    # Create DataFrame
    rename_map = {k: v for k, v in zip(model_keys, col_names)}
    df_table = df_table.rename(columns=rename_map)

    # --- Add summary rows ---
    summary_rows = []

    # Control
    control_row = {'Variables': 'Control'}
    control_row.update({col: 'YES' for col in col_names})
    summary_rows.append(control_row)

    # Fixed Effects
    fe_row1 = {'Variables': 'Province FE'}
    fe_row1.update({col: 'YES' for col in col_names})
    fe_row2 = {'Variables': 'Year FE'}
    fe_row2.update({col: 'YES' for col in col_names})
    summary_rows.extend([fe_row1, fe_row2])

    # Observations
    obs_row = {'Variables': 'Observations'}
    for key, col in zip(model_keys, col_names):
        obs_row[col] = model_data[key]['N']
    summary_rows.append(obs_row)

    # R-squared
    rsq_row = {'Variables': 'R-sq'}
    for key, col in zip(model_keys, col_names):
        r2 = R_squared(results[key], sample_info[key])
        rsq_row[col] = f"{r2:.3f}"
    summary_rows.append(rsq_row)

    # Combine
    df_summary = pd.DataFrame(summary_rows)
    df_final = pd.concat([df_table, df_summary], ignore_index=True)
    df_final = df_final[['Variables'] + col_names]

    print(df_final)
    df_final.to_excel(output_file, index=False, engine='openpyxl')
    print(f"\nRegression table exported to: {output_file}")


# ============================================
# 1. Baseline model regression results. Table1
# ============================================

In [2]:

def run_baseline_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    # --- Variable Generation ---
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']

    # models
    model_specs = {
        'm1': (['lnDiv', 'lnDiv_square'], None),
        'm2': (['lnUbi', 'lnUbi_square'], None),
        'm3': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm4': (['lnUbi', 'lnUbi_square'], df['lnDiv'] > df['lnDiv'].median()),
        'm5': (['lnUbi', 'lnUbi_square'], ~(df['lnDiv'] > df['lnDiv'].median())),
        'm6': (['lnGENEPY', 'lnGENEPY_square'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df[mask] if mask is not None else df.copy()
        all_vars = ['lnCO2'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['lnCO2']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    # Define variable display labels
    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi2',
        'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m1', 'm2', 'm3', 'm4', 'm5', 'm6'],
        col_names=['(1)', '(2)', '(3)', '(4)', '(5)', '(6)'],
        controls=controls,
        output_file=f"{data_path}/Table1_Baseline_Regression_table.xlsx"
    )

if __name__ == "__main__":
    print('=====================================================\n')
    print('1. Baseline model regression results.\n\n=====================================================\n\n')
    run_baseline_models()


1. Baseline model regression results.



                  Variables        (1)        (2)       (3)       (4)        (5)        (6)
0                     lnDiv  -0.162***              0.865**                                
1                              (0.034)              (0.435)                                
2                    lnDiv2   0.022***               -0.005                                
3                              (0.005)              (0.012)                                
4                     lnUbi             -1.600***     0.332    -0.671  -3.805***           
5                                         (0.548)   (0.804)   (2.068)    (0.944)           
6                    lnUbi2              0.332***     0.116     0.157   0.785***           
7                                         (0.110)   (0.123)   (0.415)    (0.186)           
8               ln(Div×Ubi)                        -0.766**                                
9                                     

# ==============================
# 2. Moderating (GreenPat)
# ==============================

In [3]:

def run_greenpat_moderating_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']

    # 使用 GreenPat 替代 Pat
    vars_to_center = ['lnDiv', 'lnDiv_square', 'lnUbi', 'lnUbi_square', 'lnGENEPY', 'lnGENEPY_square', 'GreenPat']
    for var in vars_to_center:
        if var in df.columns:
            df[f'c_{var}'] = df[var] - df[var].mean()

    df['c_lnDivGreenPat'] = df['c_lnDiv'] * df['c_GreenPat']
    df['c_lnDiv2GreenPat'] = df['c_lnDiv_square'] * df['c_GreenPat']
    df['c_lnUbiGreenPat'] = df['c_lnUbi'] * df['c_GreenPat']
    df['c_lnUbi2GreenPat'] = df['c_lnUbi_square'] * df['c_GreenPat']
    df['c_lnGENEPYGreenPat'] = df['c_lnGENEPY'] * df['c_GreenPat']
    df['c_lnGENEPY2GreenPat'] = df['c_lnGENEPY_square'] * df['c_GreenPat']

    model_specs = {
        'm46': (['lnDiv', 'lnDiv_square', 'c_lnDivGreenPat', 'c_lnDiv2GreenPat', 'GreenPat'], None),
        'm47': (['lnUbi', 'lnUbi_square', 'c_lnUbiGreenPat', 'c_lnUbi2GreenPat', 'GreenPat'], None),
        'm48': (['lnGENEPY', 'lnGENEPY_square', 'c_lnGENEPYGreenPat', 'c_lnGENEPY2GreenPat', 'GreenPat'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df.copy()
        all_vars = ['lnCO2'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['lnCO2']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv²',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi²',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR²',
        'GreenPat': 'GreenPat',
        'c_lnDivGreenPat': 'lnDiv×GreenPat',
        'c_lnDiv2GreenPat': 'lnDiv²×GreenPat',
        'c_lnUbiGreenPat': 'lnUbi×GreenPat',
        'c_lnUbi2GreenPat': 'lnUbi²×GreenPat',
        'c_lnGENEPYGreenPat': 'lnGPYR×GreenPat',
        'c_lnGENEPY2GreenPat': 'lnGPYR²×GreenPat',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP²',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m46', 'm47', 'm48'],
        col_names=['(7)', '(8)', '(9)'],
        controls=controls,
        output_file=f"{data_path}/Table2_regression_table_GreenPat_moderating.xlsx"
    )



if __name__ == "__main__":
    print('=====================================================\n')
    print('2.Moderating (GreenPat) regression results.\n\n=====================================================\n\n')
    run_greenpat_moderating_models()


2.Moderating (GreenPat) regression results.



                  Variables        (7)        (8)        (9)
0                     lnDiv  -0.102***                      
1                              (0.038)                      
2                    lnDiv²   0.015***                      
3                              (0.005)                      
4                     lnUbi              -1.283**           
5                                         (0.566)           
6                    lnUbi²               0.263**           
7                                         (0.113)           
8                    lnGPYR                           -0.033
9                                                    (0.027)
10                  lnGPYR²                           0.009*
11                                                   (0.005)
12                 GreenPat  -0.000***  -0.000***  -0.000***
13                             (0.000)    (0.000)    (0.000)
14           lnDiv×GreenPat    0.000*

# ============================================
# 3. Replacing the dependent variable with Carbon Emissions (IPE)
# ============================================

In [4]:
def run_replacing_the_dependent_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    # --- Variable Generation ---
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']

    # models
    model_specs = {
        'm7': (['lnDiv', 'lnDiv_square'], None),
        'm8': (['lnUbi', 'lnUbi_square'], None),
        'm9': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm10': (['lnUbi', 'lnUbi_square'], df['lnDiv'] > df['lnDiv'].median()),
        'm11': (['lnUbi', 'lnUbi_square'], ~(df['lnDiv'] > df['lnDiv'].median())),
        'm12': (['lnGENEPY', 'lnGENEPY_square'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df[mask] if mask is not None else df.copy()
        all_vars = ['lnCO2ipe'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['lnCO2ipe']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    # Define variable display labels
    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi2',
        'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m7', 'm8', 'm9', 'm10', 'm11', 'm12'],
        col_names=['(1)', '(2)', '(3)', '(4)', '(5)', '(6)'],
        controls=controls,
        output_file=f"{data_path}/TableA1_Regression_table_Replacing_the_dependent_variable_IPE_CO2.xlsx"
    )

if __name__ == "__main__":
    print('============================================================\n')
    print('3. Replacing the dependent variable with Carbon Emissions (IPE)\n\n============================================================\n\n')
    run_replacing_the_dependent_models()


3. Replacing the dependent variable with Carbon Emissions (IPE)



                  Variables        (1)        (2)        (3)       (4)        (5)       (6)
0                     lnDiv  -0.157***               1.144**                               
1                              (0.035)               (0.444)                               
2                    lnDiv2   0.021***                -0.013                               
3                              (0.005)               (0.012)                               
4                     lnUbi             -1.508***      0.778    -1.403  -3.363***          
5                                         (0.559)    (0.820)   (2.067)    (0.985)          
6                    lnUbi2              0.316***      0.071     0.307   0.700***          
7                                         (0.113)    (0.125)   (0.414)    (0.194)          
8               ln(Div×Ubi)                        -0.977***                               
9           

# ================================================
# 4. Replacing the dependent variable with Energy Structure
# ================================================

In [5]:
def run_replacing_the_dependent_energy_structure_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    # --- Variable Generation ---
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']

    # models
    model_specs = {
        'm13': (['lnDiv', 'lnDiv_square'], None),
        'm14': (['lnUbi', 'lnUbi_square'], None),
        'm15': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm16': (['lnUbi', 'lnUbi_square'], df['lnDiv'] > df['lnDiv'].median()),
        'm17': (['lnUbi', 'lnUbi_square'], ~(df['lnDiv'] > df['lnDiv'].median())),
        'm18': (['lnGENEPY', 'lnGENEPY_square'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df[mask] if mask is not None else df.copy()
        all_vars = ['Energy_Structure'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['Energy_Structure']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    # Define variable display labels
    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi2',
        'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m13', 'm14', 'm15', 'm16', 'm17', 'm18'],
        col_names=['(13)', '(14)', '(15)', '(16)', '(17)', '(18)'],
        controls=controls,
        output_file=f"{data_path}/TableA2_Regression_table_Replacing_the_dependent_variable_Energy_Structure.xlsx"
    )

if __name__ == "__main__":
    print('========================================================\n')
    print('4. Replacing the dependent variable with Energy Structure\n\n========================================================\n\n')
    run_replacing_the_dependent_energy_structure_models()


4. Replacing the dependent variable with Energy Structure



                  Variables       (13)      (14)      (15)      (16)     (17)      (18)
0                     lnDiv   0.009***               0.028                             
1                              (0.003)             (0.040)                             
2                    lnDiv2  -0.001***             -0.002*                             
3                              (0.000)             (0.001)                             
4                     lnUbi                0.045     0.049    -0.215    0.081          
5                                        (0.048)   (0.072)   (0.156)  (0.089)          
6                    lnUbi2               -0.009    -0.008     0.044   -0.017          
7                                        (0.010)   (0.011)   (0.031)  (0.018)          
8               ln(Div×Ubi)                         -0.015                             
9                                                  (0.031)

# ============================================
# 5. Added control variables
# ============================================

In [6]:

def run_add_more_control_variables_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    # --- Variable Generation ---
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization', 'lnEaDS', 'lnFDI', 'lnElectricity_Consumption', 'lnTransport_Storage_Post']

    # models
    model_specs = {
        'm49': (['lnDiv', 'lnDiv_square'], None),
        'm50': (['lnUbi', 'lnUbi_square'], None),
        'm51': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm52': (['lnUbi', 'lnUbi_square'], df['lnDiv'] > df['lnDiv'].median()),
        'm53': (['lnUbi', 'lnUbi_square'], ~(df['lnDiv'] > df['lnDiv'].median())),
        'm54': (['lnGENEPY', 'lnGENEPY_square'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df[mask] if mask is not None else df.copy()
        all_vars = ['lnCO2'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['lnCO2']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    # Define variable display labels
    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi2',
        'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization',
        'lnEaDS': 'lnIndustrial Enterprises above Designated Size', 
        'lnFDI': 'lnForeign Direct Investment',
        'lnElectricity_Consumption':'lnElectricity Consumption',
        'lnTransport_Storage_Post': 'lnTransport Storage_Post'        
        
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m49', 'm50', 'm51', 'm52', 'm53', 'm54'],
        col_names=['(1)', '(2)', '(3)', '(4)', '(5)', '(6)'],
        controls=controls,
        output_file=f"{data_path}/TableA3_Regression_table_added_more_control_variables.xlsx"
    )

if __name__ == "__main__":
    print('========================================================\n')
    print('5. Added control variables\n\n========================================================\n\n')
    run_add_more_control_variables_models()


5. Added control variables



                                         Variables        (1)        (2)        (3)       (4)        (5)        (6)
0                                            lnDiv  -0.083***                -0.051                                
1                                                     (0.025)               (0.314)                                
2                                           lnDiv2   0.011***                 0.010                                
3                                                     (0.003)               (0.009)                                
4                                            lnUbi              -0.816**     -0.527     1.266   -1.424**           
5                                                                (0.391)    (0.577)   (1.689)    (0.655)           
6                                           lnUbi2               0.172**      0.126    -0.234    0.292**           
7                                        


# ==============================
# 6. Data Trimming
# ==============================

In [7]:
def run_winsorized_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    # --- Helper: Winsorize at 2% and 98% ---
    def winsorize_series(s, p_low=2, p_high=98):
        low_val = s.quantile(p_low / 100, interpolation='lower')
        high_val = s.quantile(p_high / 100, interpolation='higher')
        return s.clip(lower=low_val, upper=high_val)

    # Winsorize main variables
    df['lnDiv'] = winsorize_series(df['lnDiv'])
    df['lnUbi'] = winsorize_series(df['lnUbi'])
    df['lnGENEPY'] = winsorize_series(df['lnGENEPY'])

    # Generate and winsorize squared terms
    df['lnDiv_square'] = winsorize_series(df['lnDiv'] ** 2)
    df['lnUbi_square'] = winsorize_series(df['lnUbi'] ** 2)
    df['lnGENEPY_square'] = winsorize_series(df['lnGENEPY'] ** 2)

    # Interaction term (from winsorized vars)
    df['lnDivUbi'] = df['lnDiv'] * df['lnUbi']

    # Control variables
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']

    # Median split for subgroup analysis
    median_lnDiv = df['lnDiv'].median()
    df['high_lnDiv'] = df['lnDiv'] > median_lnDiv

    # Model specifications
    model_specs = {
        'm25': (['lnDiv', 'lnDiv_square'], None),
        'm26': (['lnUbi', 'lnUbi_square'], None),
        'm27': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm28': (['lnUbi', 'lnUbi_square'], df['high_lnDiv']),
        'm29': (['lnUbi', 'lnUbi_square'], ~df['high_lnDiv']),
        'm30': (['lnGENEPY', 'lnGENEPY_square'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df[mask] if mask is not None else df.copy()
        all_vars = ['lnCO2'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['lnCO2']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    # Variable labels
    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi2',
        'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m25', 'm26', 'm27', 'm28', 'm29', 'm30'],
        col_names=['(1)', '(2)', '(3)', '(4)', '(5)', '(6)'],
        controls=controls,
        output_file=f"{data_path}/TableA4_Regression_table_data_trimming.xlsx"
    )

if __name__ == "__main__":
    print('=====================================================\n')
    print('6. Data trimming\n\n=====================================================\n\n')
    run_winsorized_models()


6. Data trimming



                  Variables        (1)        (2)       (3)       (4)        (5)        (6)
0                     lnDiv  -0.234***               -0.050                                
1                              (0.084)              (0.188)                                
2                    lnDiv2   0.029***             0.028***                                
3                              (0.010)              (0.010)                                
4                     lnUbi              -2.104**    -1.399   -6.873*  -4.274***           
5                                         (1.037)   (1.273)   (3.698)    (1.459)           
6                    lnUbi2               0.430**     0.375    1.390*   0.908***           
7                                         (0.210)   (0.230)   (0.737)    (0.298)           
8               ln(Div×Ubi)                          -0.062                                
9                                                   (0.055)


# ==============================
# 7. Tobit model regression
# ==============================

In [8]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm, chi2
from statsmodels.tools import add_constant

pd.set_option('display.width', 1000)

# --- Helper: A class to store Tobit results with all necessary stats ---
class TobitResults:
    def __init__(self, params, std_errors, pvalues, log_likelihood, n_obs, X_names, log_likelihood_null):
        self.params = pd.Series(params, index=X_names)
        self.std_errors = pd.Series(std_errors, index=X_names)
        self.pvalues = pd.Series(pvalues, index=X_names)
        self.log_likelihood = log_likelihood
        self.nobs = n_obs
        self.log_likelihood_null = log_likelihood_null
        
        # Calculate LR test statistics upon initialization
        self.lr_stat = 2 * (self.log_likelihood - self.log_likelihood_null)
        # Degrees of freedom = number of predictors (excluding constant)
        df = len(self.params) - 1 if 'const' in self.params.index else len(self.params)
        
        if df > 0:
            self.lr_pvalue = chi2.sf(self.lr_stat, df)
        else: # Handle case with only a constant or no predictors
            self.lr_pvalue = np.nan

# --- Helper: Tobit Log-Likelihood Function ---
def tobit_log_likelihood(params, X, y, left_censor=0):
    beta = params[:-1]
    log_sigma = params[-1]
    sigma = np.exp(log_sigma)  # enforce positivity

    y_pred = X @ beta
    censored = (y <= left_censor)
    uncensored = ~censored

    # Uncensored observations: normal density
    ll_uncensored = np.sum(norm.logpdf(y[uncensored], loc=y_pred[uncensored], scale=sigma))

    # Censored observations: P(y* <= left_censor) = Φ((left_censor - xβ)/σ)
    if np.any(censored):
        z = (left_censor - y_pred[censored]) / sigma
        ll_censored = np.sum(norm.logcdf(z))
    else:
        ll_censored = 0.0

    return -(ll_uncensored + ll_censored)

# --- Helper: Main Tobit Regression Function (Corrected and Enhanced) ---
def run_tobit(y, X, left_censor=0):
    # Convert pandas inputs to float numpy arrays to avoid TypeError
    X_vals = X.values.astype(float)
    y_vals = y.values.astype(float)

    # --- 1. Estimate Full Model ---
    initial_params_full = np.append(np.zeros(X_vals.shape[1]), np.log(y_vals.std()))
    result_full = minimize(
        tobit_log_likelihood, initial_params_full, args=(X_vals, y_vals, left_censor), method='BFGS'
    )
    log_likelihood_full = -result_full.fun
    
    # --- 2. Estimate Null (Intercept-Only) Model for LR Test and Pseudo R2 ---
    X_null = add_constant(pd.DataFrame(index=X.index), has_constant='add')
    X_vals_null = X_null.values.astype(float)
    initial_params_null = np.append(np.zeros(X_vals_null.shape[1]), np.log(y_vals.std()))
    result_null = minimize(
        tobit_log_likelihood, initial_params_null, args=(X_vals_null, y_vals, left_censor), method='BFGS'
    )
    log_likelihood_null = -result_null.fun

    # --- 3. Extract parameters and standard errors for the full model ---
    if hasattr(result_full, 'hess_inv') and np.all(np.isfinite(result_full.hess_inv)):
        hessian_inv = result_full.hess_inv
        std_errors = np.sqrt(np.diag(hessian_inv))
    else:
        std_errors = np.full(len(result_full.x), np.nan)

    params = result_full.x
    pvalues = norm.sf(np.abs(params / std_errors)) * 2

    return TobitResults(
        params=params[:-1], std_errors=std_errors[:-1], pvalues=pvalues[:-1],
        log_likelihood=log_likelihood_full,
        n_obs=len(y), X_names=X.columns,
        log_likelihood_null=log_likelihood_null
    )

# --- Build custom table with ALL variables and new summary stats ---
def build_tobit_regression_table(results, var_labels, model_keys, col_names, controls_and_fe, output_file):
    all_vars = set().union(*(res.params.index for res in results.values()))
    for var in all_vars:
        if var not in var_labels: var_labels[var] = var

    main_vars_order = [v for v in var_labels if v in all_vars and v not in controls_and_fe and v != 'const']
    control_vars_order = [v for v in controls_and_fe if 'year' not in v and 'province' not in v and v in all_vars]
    const_var = ['const'] if 'const' in all_vars else []
    ordered_vars = main_vars_order + control_vars_order + const_var

    table_rows = []
    for var in ordered_vars:
        coef_row, se_row = {'Variables': var_labels[var]}, {'Variables': ''}
        for key in model_keys:
            res = results[key]
            if var in res.params.index:
                coef_row[key] = add_stars(res.params[var], res.pvalues[var])
                se_row[key] = f"({res.std_errors[var]:.3f})"
            else:
                coef_row[key], se_row[key] = "", ""
        table_rows.extend([coef_row, se_row])
    
    df_table = pd.DataFrame(table_rows).rename(columns={k: v for k, v in zip(model_keys, col_names)})

    summary_data = {
        'Control': {col: 'YES' for col in col_names},
        'Province FE': {col: 'YES' for col in col_names},
        'Year FE': {col: 'YES' for col in col_names},
        'Observations': {col: results[key].nobs for key, col in zip(model_keys, col_names)},
        'Log-likelihood': {col: f"{results[key].log_likelihood:.3f}" for key, col in zip(model_keys, col_names)},
        'Pseudo R-sq': {col: f"{1 - (results[key].log_likelihood / results[key].log_likelihood_null):.3f}" for key, col in zip(model_keys, col_names)},
        'LR chi2': {col: f"{results[key].lr_stat:.2f}" for key, col in zip(model_keys, col_names)},
        'Prob > chi2': {col: f"{results[key].lr_pvalue:.4f}" for key, col in zip(model_keys, col_names)},
    }
    df_summary = pd.DataFrame.from_dict(summary_data, orient='index').reset_index().rename(columns={'index': 'Variables'})
    
    df_final = pd.concat([df_table, df_summary], ignore_index=True)
    df_final = df_final[['Variables'] + col_names]
    
    print(df_final.to_string())
    df_final.to_excel(output_file, index=False, engine='openpyxl')
    print(f"\nRegression table exported to: {output_file}")

def run_tobit_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    province_dummies = pd.get_dummies(df['province'], prefix='province', drop_first=True)
    year_dummies = pd.get_dummies(df['year'], prefix='year', drop_first=True)
    df = pd.concat([df, province_dummies, year_dummies], axis=1)
    
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']
    controls_and_fe = controls + list(province_dummies.columns) + list(year_dummies.columns)

    median_lnDiv = df['lnDiv'].median()
    df['high_lnDiv'] = df['lnDiv'] > median_lnDiv

    model_specs = {
        'm31': (['lnDiv', 'lnDiv_square'], None),
        'm32': (['lnUbi', 'lnUbi_square'], None),
        'm33': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm34': (['lnUbi', 'lnUbi_square'], df['high_lnDiv']),
        'm35': (['lnUbi', 'lnUbi_square'], ~df['high_lnDiv']),
        'm36': (['lnGENEPY', 'lnGENEPY_square'], None),
    }
    
    results = {}
    for key, (main_vars, mask) in model_specs.items():
        #print(f"Running model {key}...")
        df_sub = df if mask is None else df[mask]
        df_clean = df_sub[['lnCO2'] + main_vars + controls_and_fe].dropna()
        y = df_clean['lnCO2']
        X = add_constant(df_clean[main_vars + controls_and_fe])
        results[key] = run_tobit(y, X, left_censor=0)

    var_labels = {
        'const': 'Constant', 'lnDiv': 'lnDiv', 'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi', 'lnUbi_square': 'lnUbi2', 'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR', 'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP', 'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary', 'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_tobit_regression_table(
        results=results, var_labels=var_labels,
        model_keys=['m31', 'm32', 'm33', 'm34', 'm35', 'm36'],
        col_names=['(1)', '(2)', '(3)', '(4)', '(5)', '(6)'], # Relabeled columns
        controls_and_fe=controls_and_fe,
        output_file=f"{data_path}/TableA5_Tobit_Regression_Table_Final.xlsx"
    )

if __name__ == "__main__":
    print('=====================================================\n')
    print('7.Tobit model regression\n\n=====================================================\n\n')
    run_tobit_models()


7.Tobit model regression



                  Variables        (1)        (2)       (3)       (4)        (5)        (6)
0                     lnDiv  -0.161***               0.867*                                
1                              (0.036)              (0.487)                                
2                    lnDiv2   0.022***               -0.005                                
3                              (0.005)              (0.013)                                
4                     lnUbi              -1.599**     0.336    -0.661  -3.804***           
5                                         (0.765)   (0.832)   (1.912)    (1.053)           
6                    lnUbi2               0.331**     0.116     0.155   0.785***           
7                                         (0.154)   (0.120)   (0.383)    (0.208)           
8               ln(Div×Ubi)                        -0.768**                                
9                                                  


# ==============================
# 8. Lagged by one period
# ==============================

In [9]:
def run_lagged_models():
    data_path = "data"
    file_name = "instrument_target_data_result_submit.xlsx"
    df = pd.read_excel(f"{data_path}/{file_name}", sheet_name="Sheet1")
    df = df.set_index(['province', 'year'])

    # --- Variable Generation ---
    df['lnpcGDP2'] = df['lnpcGDP'] ** 2
    controls = ['lnpcGDP', 'lnpcGDP2', 'Share_Secondary', 'Urbanization_Rate', 'Fiscal_Decentralization']

    # models
    model_specs = {
        'm37': (['lnDiv', 'lnDiv_square'], None),
        'm38': (['lnUbi', 'lnUbi_square'], None),
        'm39': (['lnDiv', 'lnUbi', 'lnDiv_square', 'lnUbi_square', 'lnDivUbi'], None),
        'm40': (['lnUbi', 'lnUbi_square'], df['lnDiv'] > df['lnDiv'].median()),
        'm41': (['lnUbi', 'lnUbi_square'], ~(df['lnDiv'] > df['lnDiv'].median())),
        'm42': (['lnGENEPY', 'lnGENEPY_square'], None),
    }

    results, sample_info, model_data = {}, {}, {}
    for key, (main_vars, mask) in model_specs.items():
        df_sub = df[mask] if mask is not None else df.copy()
        all_vars = ['lnCO2_lag1'] + main_vars + controls
        df_clean = df_sub[all_vars].dropna()
        y = df_clean['lnCO2_lag1']
        X = df_clean[main_vars + controls]
        X_with_const = add_constant(X)
        mod = PanelOLS(y, X_with_const, entity_effects=True, time_effects=True)
        res = mod.fit()
        results[key] = res
        sample_info[key] = y
        model_data[key] = {'y': y, 'X': X_with_const, 'main_vars': main_vars, 'N': len(y)}

    # Define variable display labels
    var_labels = {
        'const': 'Constant',
        'lnDiv': 'lnDiv',
        'lnDiv_square': 'lnDiv2',
        'lnUbi': 'lnUbi',
        'lnUbi_square': 'lnUbi2',
        'lnDivUbi': 'ln(Div×Ubi)',
        'lnGENEPY': 'lnGPYR',
        'lnGENEPY_square': 'lnGPYR2',
        'lnpcGDP': 'lnpcGDP',
        'lnpcGDP2': 'lnpcGDP2',
        'Share_Secondary': 'Share Secondary',
        'Urbanization_Rate': 'Urbanization Rate',
        'Fiscal_Decentralization': 'Fiscal Decentralization'
    }

    build_regression_table(
        results=results,
        model_data=model_data,
        sample_info=sample_info,
        var_labels=var_labels,
        model_keys=['m37', 'm38', 'm39', 'm40', 'm41', 'm42'],
        col_names=['(1)', '(2)', '(3)', '(4)', '(5)', '(6)'],
        controls=controls,
        output_file=f"{data_path}/TableA6_Regression_table_Lagged_by_one_period.xlsx"
    )

if __name__ == "__main__":
    print('=====================================================\n')
    print('8. Lagged by one period\n\n=====================================================\n\n')
    run_lagged_models()


8. Lagged by one period



                  Variables        (1)        (2)        (3)       (4)        (5)        (6)
0                     lnDiv  -0.129***              1.847***                                
1                              (0.042)               (0.624)                                
2                    lnDiv2   0.019***               -0.031*                                
3                              (0.005)               (0.017)                                
4                     lnUbi              -1.547**      0.814     1.279  -5.160***           
5                                         (0.709)    (1.017)   (2.037)    (1.419)           
6                    lnUbi2               0.297**      0.168    -0.242   1.047***           
7                                         (0.144)    (0.151)   (0.408)    (0.287)           
8               ln(Div×Ubi)                        -1.503***                                
9                                         